# FM Fingerprint Visualization

Interactive analysis of RF fingerprints from the Metal_SDR database.

**Features:**
- Time-series plots of signal quality metrics
- Frequency distribution analysis
- Quality gate validation
- Station identification patterns

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
# Connect to database
DB_PATH = Path('../fingerprints.db')
conn = sqlite3.connect(DB_PATH)
print(f"Connected to: {DB_PATH.absolute()}")

## Load Data

In [ ]:
# Query fingerprints with capture metadata
query = """
SELECT
    c.id as capture_id,
    c.timestamp,
    c.center_freq_hz / 1e6 as center_freq_mhz,
    c.sample_rate_hz / 1e6 as sample_rate_mhz,
    c.gain_db,
    f.peak_freq_hz / 1e6 as peak_freq_mhz,
    f.freq_error_hz / 1e3 as freq_error_khz,
    f.cnr_db,
    f.bandwidth_3db_hz / 1e3 as bandwidth_khz,
    f.adjacent_rejection_db,
    f.rolloff_left_slope,
    f.rolloff_right_slope,
    f.rolloff_asymmetry,
    f.processing_time_sec
FROM captures c
JOIN fingerprints f ON c.id = f.capture_id
ORDER BY c.timestamp
"""

df = pd.read_sql_query(query, conn, parse_dates=['timestamp'])
print(f"Loaded {len(df)} fingerprints")
df.head()

## Summary Statistics

In [ ]:
# Quality gate thresholds
CNR_THRESHOLD = 18.0  # dB
ADJ_REJECTION_THRESHOLD = 15.0  # dB

# Apply quality flags
df['cnr_pass'] = df['cnr_db'] >= CNR_THRESHOLD
df['rejection_pass'] = df['adjacent_rejection_db'] >= ADJ_REJECTION_THRESHOLD
df['is_reliable'] = df['cnr_pass'] & df['rejection_pass']

print("\n=== Quality Summary ===")
print(f"Total captures: {len(df)}")
print(f"Reliable (CNR ≥ {CNR_THRESHOLD} dB): {df['cnr_pass'].sum()} ({df['cnr_pass'].mean()*100:.1f}%)")
print(f"Good rejection (≥ {ADJ_REJECTION_THRESHOLD} dB): {df['rejection_pass'].sum()} ({df['rejection_pass'].mean()*100:.1f}%)")
print(f"Overall reliable: {df['is_reliable'].sum()} ({df['is_reliable'].mean()*100:.1f}%)")

print("\n=== Metric Statistics ===")
df[['cnr_db', 'bandwidth_khz', 'adjacent_rejection_db', 'freq_error_khz']].describe()

## Time Series Plots

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# CNR over time
ax = axes[0]
ax.plot(df['timestamp'], df['cnr_db'], 'o-', alpha=0.7, label='CNR')
ax.axhline(CNR_THRESHOLD, color='r', linestyle='--', alpha=0.5, label=f'Threshold ({CNR_THRESHOLD} dB)')
ax.set_ylabel('CNR (dB)')
ax.set_title('Carrier-to-Noise Ratio Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

# Bandwidth over time
ax = axes[1]
ax.plot(df['timestamp'], df['bandwidth_khz'], 'o-', alpha=0.7, color='green')
ax.set_ylabel('Bandwidth (kHz)')
ax.set_title('3dB Bandwidth Over Time')
ax.grid(True, alpha=0.3)

# Adjacent rejection over time
ax = axes[2]
ax.plot(df['timestamp'], df['adjacent_rejection_db'], 'o-', alpha=0.7, color='orange')
ax.axhline(ADJ_REJECTION_THRESHOLD, color='r', linestyle='--', alpha=0.5, label=f'Threshold ({ADJ_REJECTION_THRESHOLD} dB)')
ax.set_ylabel('Rejection (dB)')
ax.set_xlabel('Time')
ax.set_title('Adjacent Channel Rejection Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Frequency Distribution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# CNR histogram
ax = axes[0, 0]
ax.hist(df['cnr_db'], bins=30, alpha=0.7, color='blue', edgecolor='black')
ax.axvline(CNR_THRESHOLD, color='r', linestyle='--', linewidth=2, label=f'Threshold ({CNR_THRESHOLD} dB)')
ax.set_xlabel('CNR (dB)')
ax.set_ylabel('Count')
ax.set_title('CNR Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Bandwidth histogram
ax = axes[0, 1]
ax.hist(df['bandwidth_khz'], bins=30, alpha=0.7, color='green', edgecolor='black')
ax.set_xlabel('Bandwidth (kHz)')
ax.set_ylabel('Count')
ax.set_title('3dB Bandwidth Distribution')
ax.grid(True, alpha=0.3)

# Adjacent rejection histogram
ax = axes[1, 0]
ax.hist(df['adjacent_rejection_db'], bins=30, alpha=0.7, color='orange', edgecolor='black')
ax.axvline(ADJ_REJECTION_THRESHOLD, color='r', linestyle='--', linewidth=2, label=f'Threshold ({ADJ_REJECTION_THRESHOLD} dB)')
ax.set_xlabel('Adjacent Rejection (dB)')
ax.set_ylabel('Count')
ax.set_title('Adjacent Channel Rejection Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Frequency error histogram
ax = axes[1, 1]
ax.hist(df['freq_error_khz'], bins=30, alpha=0.7, color='purple', edgecolor='black')
ax.set_xlabel('Frequency Error (kHz)')
ax.set_ylabel('Count')
ax.set_title('Frequency Error Distribution')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Correlation Analysis

In [ ]:
# Select numeric columns for correlation
corr_cols = ['cnr_db', 'bandwidth_khz', 'adjacent_rejection_db', 
             'rolloff_asymmetry', 'freq_error_khz', 'gain_db']

correlation_matrix = df[corr_cols].corr()

plt.figure(figsize=(10, 8))
plt.imshow(correlation_matrix, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
plt.colorbar(label='Correlation Coefficient')
plt.xticks(range(len(corr_cols)), corr_cols, rotation=45, ha='right')
plt.yticks(range(len(corr_cols)), corr_cols)
plt.title('Feature Correlation Matrix')

# Add correlation values as text
for i in range(len(corr_cols)):
    for j in range(len(corr_cols)):
        text = plt.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                       ha='center', va='center', color='black', fontsize=10)

plt.tight_layout()
plt.show()

## Station Identification (Peak Frequency Clustering)

In [ ]:
# Group by rounded frequency (to nearest 100 kHz)
df['station_freq'] = (df['peak_freq_mhz'] * 10).round() / 10

station_stats = df.groupby('station_freq').agg({
    'capture_id': 'count',
    'cnr_db': ['mean', 'std'],
    'bandwidth_khz': 'mean',
    'adjacent_rejection_db': 'mean'
}).round(2)

station_stats.columns = ['count', 'cnr_mean', 'cnr_std', 'bw_mean', 'adj_rejection_mean']
station_stats = station_stats.sort_values('count', ascending=False)

print("\n=== Station Statistics ===")
print(station_stats.head(10))

## Quality Scatter Plots

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CNR vs Bandwidth (colored by reliability)
ax = axes[0]
reliable = df[df['is_reliable']]
unreliable = df[~df['is_reliable']]

ax.scatter(unreliable['bandwidth_khz'], unreliable['cnr_db'], 
          alpha=0.6, c='red', label='Unreliable', s=50)
ax.scatter(reliable['bandwidth_khz'], reliable['cnr_db'], 
          alpha=0.6, c='green', label='Reliable', s=50)
ax.axhline(CNR_THRESHOLD, color='orange', linestyle='--', alpha=0.5)
ax.set_xlabel('Bandwidth (kHz)')
ax.set_ylabel('CNR (dB)')
ax.set_title('CNR vs Bandwidth (Quality Gates)')
ax.legend()
ax.grid(True, alpha=0.3)

# CNR vs Adjacent Rejection
ax = axes[1]
ax.scatter(unreliable['adjacent_rejection_db'], unreliable['cnr_db'], 
          alpha=0.6, c='red', label='Unreliable', s=50)
ax.scatter(reliable['adjacent_rejection_db'], reliable['cnr_db'], 
          alpha=0.6, c='green', label='Reliable', s=50)
ax.axhline(CNR_THRESHOLD, color='orange', linestyle='--', alpha=0.5, label=f'CNR={CNR_THRESHOLD} dB')
ax.axvline(ADJ_REJECTION_THRESHOLD, color='purple', linestyle='--', alpha=0.5, label=f'Rejection={ADJ_REJECTION_THRESHOLD} dB')
ax.set_xlabel('Adjacent Rejection (dB)')
ax.set_ylabel('CNR (dB)')
ax.set_title('CNR vs Adjacent Rejection (Quality Gates)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Export Results

In [ ]:
# Export to CSV for further analysis
output_file = Path('../analysis_results.csv')
df.to_csv(output_file, index=False)
print(f"Results exported to: {output_file.absolute()}")

In [ ]:
# Close database connection
conn.close()